<a href="https://colab.research.google.com/github/ruzbro/notebooks/blob/master/Dot_Density_Dashboard_Notebooks_2020_Census_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyomnisci geopandas pandas shapely pyshp


     |████████████████████████████████| 6.4 MB 24.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.0 MB 39.5 MB/s 
     |████████████████████████████████| 219 kB 54.7 MB/s 
     |████████████████████████████████| 75 kB 4.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 168 kB 64.4 MB/s 
     |████████████████████████████████| 59 kB 7.2 MB/s 
     |████████████████████████████████| 15.4 MB 39 kB/s 
     |████████████████████████████████| 6.3 MB 38.0 MB/s 
  Created wheel for pyomnisci: filename=pyomnisci-0.27.0-py3-none-any.whl size=21002 sha256=01404e8360dc82d74dacbfe5fd02dc27d41c694880cb749cca825c865a5fc741
  Stored in directory: /root/.cache/pip/wheels/cb/d4/c8/99361452c06c38d506e85ecfd6b65aa58

In [ ]:
from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO, StringIO
import shapefile
import geopandas as gpd
from shapely.geometry import shape  
import osr
import pandas as pd
import requests
from shapely.geometry import Point
from numpy.random import RandomState, uniform
import numpy as np
import random

In [ ]:
from pyomnisci import connect

con = connect(user="admin", password="password", host="0.0.0.0", dbname="omnisci")
con


In [ ]:
def dot_density_and_pushtoomnisci(df,year,pts_per_person):
  gdf = gpd.GeoDataFrame(df, crs = "EPSG:4326", geometry=df['geometry'])

  epsg = 4326 

  list_of_point_categories = []
  for field in ['white_pop','black_pop'	,'hispanic_pop'	,'asian_pop',	'amerindian_pop',	'other_race_pop',	'two_or_more_races_pop']:
    
    ps = gpd.GeoDataFrame(gen_points_in_gdf_polys(geometry = gdf['geometry'], values=gdf[field],
                                points_per_value = pts_per_person))
    ps['ethnicity'] = field
    ps["year"] = year

    list_of_point_categories.append(ps)

  all_points=gpd.GeoDataFrame(pd.concat(list_of_point_categories))

  all_points=all_points.reset_index(drop=True)

  con = connect(user="admin", password="password", host="0.0.0.0", dbname="omnisci")
  con

  try:
      con.load_table_columnar("tablename", all_points)
              
  except Exception as e:
      print(f"Fail to insert data {e}")
        




In [ ]:
seed = 10
s=RandomState(seed) if seed else RandomState(seed)

def gen_random_points_poly(poly, num_points):
    """
    Returns a list of N randomly generated points within a polygon. 
    """
    
    min_x, min_y, max_x, max_y = poly.bounds
    points = []
    i=0
    while len(points) < num_points:
        random_point = Point([s.uniform(min_x, max_x), s.uniform(min_y, max_y)])
        if random_point.within(poly):
            points.append(random_point)
        i+=1
    return points

def gen_points_in_gdf_polys(geometry, values, points_per_value = None):
    """
    Take a GeoSeries of Polygons along with a Series of values and returns randomly generated points within
    these polygons. Optionally takes a "points_per_value" integer which indicates the number of points that 
    should be generated for each 1 value.
    """
    if points_per_value:
        new_values = (values/points_per_value).astype(int)
    else:
        new_values = values

    
    new_values = new_values[new_values>0]
    
    if(new_values.size > 0):
        g = gpd.GeoDataFrame(data = {'vals':new_values}, geometry = geometry)
        
        a = g.apply(lambda row: tuple(gen_random_points_poly(row['geometry'], row['vals'])),1)
        b = gpd.GeoSeries(a.apply(pd.Series).stack(), crs = geometry.crs)
        b.name='geometry'
      
        return b

    

In [ ]:
# You'd need to sign up for Redistricting Hub and get the state block group data files from them.

LISTOFSTATEZIPLINKS = ["ak_pl2020_bg.zip","al_pl2020_bg.zip","ar_pl2020_bg.zip","az_pl2020_bg.zip","ca_pl2020_bg.zip","co_pl2020_bg.zip","ct_pl2020_bg.zip","de_pl2020_bg.zip","ga_pl2020_bg.zip","hi_pl2020_bg.zip","ia_pl2020_bg.zip","id_pl2020_bg.zip","il_pl2020_bg.zip","in_pl2020_bg.zip","ks_pl2020_bg.zip","ky_pl2020_bg.zip","la_pl2020_bg.zip","ma_pl2020_bg.zip","md_pl2020_bg.zip","me_pl2020_bg.zip","mi_pl2020_bg.zip","mn_pl2020_bg.zip","mo_pl2020_bg.zip","ms_pl2020_bg.zip","mt_pl2020_bg.zip","nc_pl2020_bg.zip","nd_pl2020_bg.zip","ne_pl2020_bg.zip","nh_pl2020_bg.zip","nj_pl2020_bg.zip","nm_pl2020_bg.zip","nv_pl2020_bg.zip","ny_pl2020_bg.zip","oh_pl2020_bg.zip","ok_pl2020_bg.zip","or_pl2020_bg.zip","pa_pl2020_bg.zip","ri_pl2020_bg.zip","sc_pl2020_bg.zip","sd_pl2020_bg.zip","tn_pl2020_bg.zip","tx_pl2020_bg.zip","ut_pl2020_bg.zip","va_pl2020_bg.zip","vt_pl2020_bg.zip","wa_pl2020_bg.zip","wi_pl2020_bg.zip","wv_pl2020_bg.zip","wy_pl2020_bg.zip"]
for ziplink in LISTOFSTATEZIPLINKS:
  url = 'https://your-s3.amazonaws.com/' + ziplink

  gdf = gpd.read_file(url).to_crs(epsg=4326)
  gdf = gdf[['GEOID',"P0020005","P0020006","P0020007","P0020008","P0020010","P0020011","P0020002","P0020001",'geometry']] 
  
  gdf.rename(columns={  'P0020005':'white_pop',
  'P0020006':'black_pop',
  'P0020007':'amerindian_pop',
  'P0020008':'asian_pop',
  'P0020010':'other_race_pop',
  'P0020011':'two_or_more_races_pop',
  'P0020002':'hispanic_pop',
  'P0020001':'total_pop'
  }, inplace=True)
  print(gdf.head())

  gdf["white_pop"] = pd.to_numeric(gdf["white_pop"])
  gdf["black_pop"] = pd.to_numeric(gdf["black_pop"])
  gdf["hispanic_pop"] = pd.to_numeric(gdf["hispanic_pop"])
  gdf["asian_pop"] = pd.to_numeric(gdf["asian_pop"])
  gdf["amerindian_pop"] = pd.to_numeric(gdf["amerindian_pop"])
  gdf["other_race_pop"] = pd.to_numeric(gdf["other_race_pop"])
  gdf["two_or_more_races_pop"] = pd.to_numeric(gdf["two_or_more_races_pop"])
  gdf["total_pop"] = pd.to_numeric(gdf["total_pop"])

  dot_density_and_pushtoomnisci(gdf,'2020-01-01T00:12:00.000Z',1)

  

URLError: ignored